In [1]:
class Node:
    
    def __init__(self, name, parents, cpt):
        self.name = name
        self.parents = parents
        self.cpt = cpt

    def __str__(self):
        return self.name

In [26]:

class CustomBayesModel:

    def __init__(self, nodes):
        self.nodes = nodes
    
    def get_node(self, name):
        for node in self.nodes:
            if node.name == name:
                return node
        return None
    
    def get_parents(self, node):
        parents = []
        for parent in node.parents:
            parents.append(self.get_node(parent))
        return parents
    
    def get_children(self, node):
        children = []
        for child in self.nodes:
            if node.name in child.parents:
                children.append(child)
        return children
    
    def add_edge(self, parent, child):
        child.parents.append(parent.name)
    
    def remove_edge(self, parent, child):
        child.parents.remove(parent.name)
    
    def get_ancestors(self, node):
        ancestors = []
        parents = self.get_parents(node)
        for parent in parents:
            ancestors.append(parent)
            ancestors.extend(self.get_ancestors(parent))
        return ancestors
    
    def get_descendants(self, node):
        descendants = []
        children = self.get_children(node)
        for child in children:
            descendants.append(child)
            descendants.extend(self.get_descendants(child))
        return descendants
    
    def get_independent_nodes(self, node):
        independent_nodes = []
        for node in self.nodes:
            if node not in self.get_ancestors(node) and node not in self.get_descendants(node):
                independent_nodes.append(node)
        return independent_nodes
    
    def get_conditional_probability(self, node, evidence):
        parents = self.get_parents(node)
        if len(parents) == 0:
            return node.cpt[0][1]
        else:
            for row in node.cpt:
                if row[0] == evidence:
                    return row[1]
        return None
    
    def get_probability(self, node, evidence):
        if node.name in evidence:
            for row in node.cpt:
                if row[0] == evidence[node.name]:
                    return row[1]
        return None

    def get_probability_of_evidence_given(self, evidence, node):
        probability = 1
        for parent in self.get_parents(node):
            if parent.name in evidence:
                probability *= self.get_probability(parent, evidence)
        return probability
    
    
    # Inference method which will conclude if PlayTennis is Yes or No based on the given evidence
    def inference(self, evidence):
        probability_yes = self.get_probability_of_evidence_given(evidence, self.get_node('PlayTennis'))
        probability_no = 1 - probability_yes

        print(probability_yes)

        return 'Yes' if probability_yes > probability_no else 'No'


In [27]:
newBayesModel = CustomBayesModel([
    Node('Outlook', [], [
        [['Sunny'], 5/14],
        [['Overcast'], 4/14],
        [['Rain'], 5/14]
    ]),
    Node('Temperature', [], [
        [['Hot'], 4/14],
        [['Mild'], 6/14],
        [['Cool'], 4/14]
    ]),
    Node('Humidity', [], [
        [['High'], 7/14],
        [['Normal'], 7/14]
    ]),
    Node('Wind', [], [
        [['Weak'], 8/14],
        [['Strong'], 6/14]
    ]),
    Node('PlayTennis', ['Outlook', 'Temperature', 'Humidity', 'Wind'], [
        [['Sunny', 'Hot', 'High', 'Weak', 'Yes'], 2/9],
        [['Sunny', 'Hot', 'High', 'Strong', 'No'], 0/9],
        [['Overcast', 'Hot', 'High', 'Weak', 'Yes'], 4/9],
        [['Rain', 'Mild', 'High', 'Weak', 'Yes'], 3/9],
        [['Rain', 'Cool', 'Normal', 'Weak', 'Yes'], 2/9],
        [['Rain', 'Cool', 'Normal', 'Strong', 'No'], 0/9],
        [['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'], 4/9],
        [['Sunny', 'Mild', 'High', 'Weak', 'No'], 0/9],
        [['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'], 3/9],
        [['Rain', 'Mild', 'Normal', 'Weak', 'Yes'], 2/9],
        [['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'], 6/9],
        [['Overcast', 'Mild', 'High', 'Strong', 'Yes'], 4/9],
        [['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'], 4/9],
        [['Rain', 'Mild', 'High', 'Strong', 'No'], 0/9],
        [['Yes'],10/14],
        [['No'], 4/14]
    ])
])

In [28]:
print(newBayesModel.inference({
    'Outlook': 'Sunny',
    'Temperature': 'Cool',
    'Humidity': 'Normal',
    'Wind': 'Strong'
}), '\n')

print(newBayesModel.inference({
    'Outlook': 'Overcast',
    'Temperature': 'Cool',
    'Humidity': 'Normal',
    'Wind': 'Strong'
}), '\n')

print(newBayesModel.inference({
    'Outlook': 'Rain',
    'Temperature': 'Cool',
    'Humidity': 'Normal',
    'Wind': 'Strong'
}), '\n')

TypeError: unsupported operand type(s) for *=: 'int' and 'NoneType'